In [283]:
import matplotlib.pyplot as plt
import sys
import numpy as np
import pandas as pd 
import netCDF4 as nc
from cartopy import config
import cartopy.crs as ccrs
import os
import shutil
import tempfile

In [284]:
np.set_printoptions(threshold=sys.maxsize)
os.chdir(r"C:\Users\eliza\Documents\Uni\Project\Data")

minlat = 0  # Set Latitude bounds (90 - 180 is NH, 0 - 90 is SH)
maxlat = 90  
minlong = 0  # Set Longitude Bounds
maxlong = 360  # 0-360 for latitude

## 1984

In [285]:
#1984
a = -1  # counter for Jun
b = 123  # counter for Jul
c = 247  # counter for Aug

WS_all = np.zeros((373,181,360))  # Stores JJA for eaach year
WSX_all = np.zeros((373,181,360))

In [286]:
fn = r"WS\1984.nc"  # Select WS data for each year
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360) # Set the axes
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jun = ds.variables['june'][:]  # Identify correct dataset for each month
jun6hr = jun[0::2]  # Make data 6 hourly
jun6hr = np.ma.filled(jun6hr, fill_value=0)  # Fill in masked data
juntranslated = np.roll(jun6hr, 180, axis=2) # Translate WS data by 180 degrees
junsize = np.size(jun6hr, 0)

jul = ds.variables['july'][:]
jul6hr = jul[0::2]
jul6hr = np.ma.filled(jul6hr, fill_value=0)
jultranslated = np.roll(jul6hr, 180, axis=2)
julsize = np.size(jul6hr, 0)

aug = ds.variables['august'][:]
aug6hr = aug[0::2]
aug6hr = np.ma.filled(aug6hr, fill_value=0)
augtranslated = np.roll(aug6hr, 180, axis=2)
augsize = np.size(aug6hr, 0)

In [287]:
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1984\JUN"):  # Run through the fronts files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1984\JUN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # Empty array for storing WS data
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))  # Shape must match cdi

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:  # Selects gridboxes containing matched fronts
                    WS[i,j] = juntranslated[a,j,i]                    
                elif ct[i,j] != 0:  # Selects unmatched fronts
                    WSX[i,j] = juntranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX


In [288]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1984\JUL"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1984\JUL", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jultranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jultranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [289]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1984\AUG"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1984\AUG", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = augtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = augtranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [290]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [291]:
WS1984 = np.zeros(11)  # Count occurences of each WS for with DI
WS1984[0] = ws.count(1)
WS1984[1] = ws.count(2)
WS1984[2] = ws.count(3)
WS1984[3] = ws.count(4)
WS1984[4] = ws.count(5)
WS1984[5] = ws.count(6)
WS1984[6] = ws.count(7)
WS1984[7] = ws.count(8)
WS1984[8] = ws.count(9)
WS1984[9] = ws.count(10)
WS1984[10] = ws.count(11)

WS1984x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1984x[0] = wsx.count(1)
WS1984x[1] = wsx.count(2)
WS1984x[2] = wsx.count(3)
WS1984x[3] = wsx.count(4)
WS1984x[4] = wsx.count(5)
WS1984x[5] = wsx.count(6)
WS1984x[6] = wsx.count(7)
WS1984x[7] = wsx.count(8)
WS1984x[8] = wsx.count(9)
WS1984x[9] = wsx.count(10)
WS1984x[10] = wsx.count(11)

print(WS1984, WS1984x)

[212. 674.  44.  76. 260.  25. 602. 101. 171.  64.   0.] [ 179. 2496.  243.  541. 1289.   95. 2865.  705.  730.  253.   16.]


## 1985

In [292]:
#1985
a = -1  # counter for Jun
b = 123  # counter for Jul
c = 247  # counter for Aug

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [293]:
fn = r"WS\1985.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jun = ds.variables['june'][:]
jun6hr = jun[0::2]
jun6hr = np.ma.filled(jun6hr, fill_value=0)
juntranslated = np.roll(jun6hr, 180, axis=2)
junsize = np.size(jun6hr, 0)

jul = ds.variables['july'][:]
jul6hr = jul[0::2]
jul6hr = np.ma.filled(jul6hr, fill_value=0)
jultranslated = np.roll(jul6hr, 180, axis=2)
julsize = np.size(jul6hr, 0)

aug = ds.variables['august'][:]
aug6hr = aug[0::2]
aug6hr = np.ma.filled(aug6hr, fill_value=0)
augtranslated = np.roll(aug6hr, 180, axis=2)
augsize = np.size(aug6hr, 0)

In [294]:
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1985\JUN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1985\JUN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = juntranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = juntranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [295]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1985\JUL"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1985\JUL", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jultranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jultranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [296]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1985\AUG"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1985\AUG", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = augtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = augtranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [297]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [298]:
WS1985 = np.zeros(11)  # Count occurences of each WS for with DI
WS1985[0] = ws.count(1)
WS1985[1] = ws.count(2)
WS1985[2] = ws.count(3)
WS1985[3] = ws.count(4)
WS1985[4] = ws.count(5)
WS1985[5] = ws.count(6)
WS1985[6] = ws.count(7)
WS1985[7] = ws.count(8)
WS1985[8] = ws.count(9)
WS1985[9] = ws.count(10)
WS1985[10] = ws.count(11)

WS1985x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1985x[0] = wsx.count(1)
WS1985x[1] = wsx.count(2)
WS1985x[2] = wsx.count(3)
WS1985x[3] = wsx.count(4)
WS1985x[4] = wsx.count(5)
WS1985x[5] = wsx.count(6)
WS1985x[6] = wsx.count(7)
WS1985x[7] = wsx.count(8)
WS1985x[8] = wsx.count(9)
WS1985x[9] = wsx.count(10)
WS1985x[10] = wsx.count(11)

print(WS1985, WS1985x)

[218. 627.  48.  37. 173.   5. 233. 197. 103.  44.   2.] [ 516. 2528.  526.  241.  951.  108. 2089. 1557.  563.  140.   14.]


## 1986

In [299]:
#1986
a = -1  # counter for Jun
b = 123  # counter for Jul
c = 247  # counter for Aug

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [300]:
fn = r"WS\1986.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jun = ds.variables['june'][:]
jun6hr = jun[0::2]
jun6hr = np.ma.filled(jun6hr, fill_value=0)
juntranslated = np.roll(jun6hr, 180, axis=2)
junsize = np.size(jun6hr, 0)

jul = ds.variables['july'][:]
jul6hr = jul[0::2]
jul6hr = np.ma.filled(jul6hr, fill_value=0)
jultranslated = np.roll(jul6hr, 180, axis=2)
julsize = np.size(jul6hr, 0)

aug = ds.variables['august'][:]
aug6hr = aug[0::2]
aug6hr = np.ma.filled(aug6hr, fill_value=0)
augtranslated = np.roll(aug6hr, 180, axis=2)
augsize = np.size(aug6hr, 0)

In [301]:
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1986\JUN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1986\JUN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = juntranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = juntranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [302]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1986\JUL"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1986\JUL", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jultranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jultranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [303]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1986\AUG"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1986\AUG", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = augtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = augtranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [304]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [305]:
WS1986 = np.zeros(11)  # Count occurences of each WS for with DI
WS1986[0] = ws.count(1)
WS1986[1] = ws.count(2)
WS1986[2] = ws.count(3)
WS1986[3] = ws.count(4)
WS1986[4] = ws.count(5)
WS1986[5] = ws.count(6)
WS1986[6] = ws.count(7)
WS1986[7] = ws.count(8)
WS1986[8] = ws.count(9)
WS1986[9] = ws.count(10)
WS1986[10] = ws.count(11)

WS1986x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1986x[0] = wsx.count(1)
WS1986x[1] = wsx.count(2)
WS1986x[2] = wsx.count(3)
WS1986x[3] = wsx.count(4)
WS1986x[4] = wsx.count(5)
WS1986x[5] = wsx.count(6)
WS1986x[6] = wsx.count(7)
WS1986x[7] = wsx.count(8)
WS1986x[8] = wsx.count(9)
WS1986x[9] = wsx.count(10)
WS1986x[10] = wsx.count(11)

print(WS1986, WS1986x)

[153. 804.  54.  47.  89.   7. 228. 148.  41.   8.   0.] [ 571. 2804.  479.  318. 1049.  103. 2009. 1498.  343.  176.   13.]


## 1987

In [306]:
#1987
a = -1  # counter for Jun
b = 123  # counter for Jul
c = 247  # counter for Aug

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [307]:
fn = r"WS\1987.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jun = ds.variables['june'][:]
jun6hr = jun[0::2]
jun6hr = np.ma.filled(jun6hr, fill_value=0)
juntranslated = np.roll(jun6hr, 180, axis=2)
junsize = np.size(jun6hr, 0)

jul = ds.variables['july'][:]
jul6hr = jul[0::2]
jul6hr = np.ma.filled(jul6hr, fill_value=0)
jultranslated = np.roll(jul6hr, 180, axis=2)
julsize = np.size(jul6hr, 0)

aug = ds.variables['august'][:]
aug6hr = aug[0::2]
aug6hr = np.ma.filled(aug6hr, fill_value=0)
augtranslated = np.roll(aug6hr, 180, axis=2)
augsize = np.size(aug6hr, 0)

In [308]:
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1987\JUN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1987\JUN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = juntranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = juntranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [309]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1987\JUL"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1987\JUL", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jultranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jultranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [310]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1987\AUG"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1987\AUG", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = augtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = augtranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [311]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [312]:
WS1987 = np.zeros(11)  # Count occurences of each WS for with DI
WS1987[0] = ws.count(1)
WS1987[1] = ws.count(2)
WS1987[2] = ws.count(3)
WS1987[3] = ws.count(4)
WS1987[4] = ws.count(5)
WS1987[5] = ws.count(6)
WS1987[6] = ws.count(7)
WS1987[7] = ws.count(8)
WS1987[8] = ws.count(9)
WS1987[9] = ws.count(10)
WS1987[10] = ws.count(11)

WS1987x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1987x[0] = wsx.count(1)
WS1987x[1] = wsx.count(2)
WS1987x[2] = wsx.count(3)
WS1987x[3] = wsx.count(4)
WS1987x[4] = wsx.count(5)
WS1987x[5] = wsx.count(6)
WS1987x[6] = wsx.count(7)
WS1987x[7] = wsx.count(8)
WS1987x[8] = wsx.count(9)
WS1987x[9] = wsx.count(10)
WS1987x[10] = wsx.count(11)

print(WS1987, WS1987x)

[185. 738.  49.  39. 168.  32. 253. 169.  37.  16.   1.] [ 734. 3367.  407.  319. 1112.   79. 2293. 1995.  448.  246.   21.]


## 1988

In [313]:
#1988
a = -1  # counter for Jun
b = 123  # counter for Jul
c = 247  # counter for Aug

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [314]:
fn = r"WS\1988.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jun = ds.variables['june'][:]
jun6hr = jun[0::2]
jun6hr = np.ma.filled(jun6hr, fill_value=0)
juntranslated = np.roll(jun6hr, 180, axis=2)
junsize = np.size(jun6hr, 0)

jul = ds.variables['july'][:]
jul6hr = jul[0::2]
jul6hr = np.ma.filled(jul6hr, fill_value=0)
jultranslated = np.roll(jul6hr, 180, axis=2)
julsize = np.size(jul6hr, 0)

aug = ds.variables['august'][:]
aug6hr = aug[0::2]
aug6hr = np.ma.filled(aug6hr, fill_value=0)
augtranslated = np.roll(aug6hr, 180, axis=2)
augsize = np.size(aug6hr, 0)

In [315]:
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1988\JUN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1988\JUN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = juntranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = juntranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [316]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1988\JUL"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1988\JUL", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jultranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jultranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [317]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1988\AUG"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1988\AUG", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = augtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = augtranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [318]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [319]:
WS1988 = np.zeros(11)  # Count occurences of each WS for with DI
WS1988[0] = ws.count(1)
WS1988[1] = ws.count(2)
WS1988[2] = ws.count(3)
WS1988[3] = ws.count(4)
WS1988[4] = ws.count(5)
WS1988[5] = ws.count(6)
WS1988[6] = ws.count(7)
WS1988[7] = ws.count(8)
WS1988[8] = ws.count(9)
WS1988[9] = ws.count(10)
WS1988[10] = ws.count(11)

WS1988x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1988x[0] = wsx.count(1)
WS1988x[1] = wsx.count(2)
WS1988x[2] = wsx.count(3)
WS1988x[3] = wsx.count(4)
WS1988x[4] = wsx.count(5)
WS1988x[5] = wsx.count(6)
WS1988x[6] = wsx.count(7)
WS1988x[7] = wsx.count(8)
WS1988x[8] = wsx.count(9)
WS1988x[9] = wsx.count(10)
WS1988x[10] = wsx.count(11)

print(WS1988, WS1988x)

[ 96. 392.  38.  27. 140.   1. 276. 193.  53.  25.   0.] [ 325. 1678.  347.  265.  626.   93. 1273. 1184.  260.   83.   13.]


## 1989

In [320]:
#1989
a = -1  # counter for Jun
b = 123  # counter for Jul
c = 247  # counter for Aug

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [321]:
fn = r"WS\1989.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jun = ds.variables['june'][:]
jun6hr = jun[0::2]
jun6hr = np.ma.filled(jun6hr, fill_value=0)
juntranslated = np.roll(jun6hr, 180, axis=2)
junsize = np.size(jun6hr, 0)

jul = ds.variables['july'][:]
jul6hr = jul[0::2]
jul6hr = np.ma.filled(jul6hr, fill_value=0)
jultranslated = np.roll(jul6hr, 180, axis=2)
julsize = np.size(jul6hr, 0)

aug = ds.variables['august'][:]
aug6hr = aug[0::2]
aug6hr = np.ma.filled(aug6hr, fill_value=0)
augtranslated = np.roll(aug6hr, 180, axis=2)
augsize = np.size(aug6hr, 0)

In [322]:
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1989\JUN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1989\JUN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = juntranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = juntranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [323]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1989\JUL"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1989\JUL", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jultranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jultranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [324]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1989\AUG"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1989\AUG", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = augtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = augtranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [325]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [326]:
WS1989 = np.zeros(11)  # Count occurences of each WS for with DI
WS1989[0] = ws.count(1)
WS1989[1] = ws.count(2)
WS1989[2] = ws.count(3)
WS1989[3] = ws.count(4)
WS1989[4] = ws.count(5)
WS1989[5] = ws.count(6)
WS1989[6] = ws.count(7)
WS1989[7] = ws.count(8)
WS1989[8] = ws.count(9)
WS1989[9] = ws.count(10)
WS1989[10] = ws.count(11)

WS1989x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1989x[0] = wsx.count(1)
WS1989x[1] = wsx.count(2)
WS1989x[2] = wsx.count(3)
WS1989x[3] = wsx.count(4)
WS1989x[4] = wsx.count(5)
WS1989x[5] = wsx.count(6)
WS1989x[6] = wsx.count(7)
WS1989x[7] = wsx.count(8)
WS1989x[8] = wsx.count(9)
WS1989x[9] = wsx.count(10)
WS1989x[10] = wsx.count(11)

print(WS1989, WS1989x)

[251. 797.  14.  69. 153.   2. 366. 260.  44.  27.   0.] [ 291. 2926.  228.  491. 1168.   99. 2254. 1712.  312.  155.   40.]


## 1990

In [327]:
#1990
a = -1  # counter for Jun
b = 123  # counter for Jul
c = 247  # counter for Aug

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [328]:
fn = r"WS\1990.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jun = ds.variables['june'][:]
jun6hr = jun[0::2]
jun6hr = np.ma.filled(jun6hr, fill_value=0)
juntranslated = np.roll(jun6hr, 180, axis=2)
junsize = np.size(jun6hr, 0)

jul = ds.variables['july'][:]
jul6hr = jul[0::2]
jul6hr = np.ma.filled(jul6hr, fill_value=0)
jultranslated = np.roll(jul6hr, 180, axis=2)
julsize = np.size(jul6hr, 0)

aug = ds.variables['august'][:]
aug6hr = aug[0::2]
aug6hr = np.ma.filled(aug6hr, fill_value=0)
augtranslated = np.roll(aug6hr, 180, axis=2)
augsize = np.size(aug6hr, 0)

In [329]:
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1990\JUN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1990\JUN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = juntranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = juntranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [330]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1990\JUL"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1990\JUL", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jultranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jultranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [331]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1990\AUG"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1990\AUG", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = augtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = augtranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [332]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [333]:
WS1990 = np.zeros(11)  # Count occurences of each WS for with DI
WS1990[0] = ws.count(1)
WS1990[1] = ws.count(2)
WS1990[2] = ws.count(3)
WS1990[3] = ws.count(4)
WS1990[4] = ws.count(5)
WS1990[5] = ws.count(6)
WS1990[6] = ws.count(7)
WS1990[7] = ws.count(8)
WS1990[8] = ws.count(9)
WS1990[9] = ws.count(10)
WS1990[10] = ws.count(11)

WS1990x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1990x[0] = wsx.count(1)
WS1990x[1] = wsx.count(2)
WS1990x[2] = wsx.count(3)
WS1990x[3] = wsx.count(4)
WS1990x[4] = wsx.count(5)
WS1990x[5] = wsx.count(6)
WS1990x[6] = wsx.count(7)
WS1990x[7] = wsx.count(8)
WS1990x[8] = wsx.count(9)
WS1990x[9] = wsx.count(10)
WS1990x[10] = wsx.count(11)

print(WS1990, WS1990x)

[ 94. 413.  23.  25. 139.   4. 232. 178.  41.  31.   6.] [ 586. 3121.  199.  233. 1040.   52. 1962. 1892.  408.  209.    9.]


## 1991

In [334]:
#1991
a = -1  # counter for Jun
b = 123  # counter for Jul
c = 247  # counter for Aug

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [335]:
fn = r"WS\1991.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jun = ds.variables['june'][:]
jun6hr = jun[0::2]
jun6hr = np.ma.filled(jun6hr, fill_value=0)
juntranslated = np.roll(jun6hr, 180, axis=2)
junsize = np.size(jun6hr, 0)

jul = ds.variables['july'][:]
jul6hr = jul[0::2]
jul6hr = np.ma.filled(jul6hr, fill_value=0)
jultranslated = np.roll(jul6hr, 180, axis=2)
julsize = np.size(jul6hr, 0)

aug = ds.variables['august'][:]
aug6hr = aug[0::2]
aug6hr = np.ma.filled(aug6hr, fill_value=0)
augtranslated = np.roll(aug6hr, 180, axis=2)
augsize = np.size(aug6hr, 0)

In [336]:
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1991\JUN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1991\JUN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = juntranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = juntranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [337]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1991\JUL"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1991\JUL", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jultranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jultranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [338]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1991\AUG"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1991\AUG", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = augtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = augtranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [339]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [340]:
WS1991 = np.zeros(11)  # Count occurences of each WS for with DI
WS1991[0] = ws.count(1)
WS1991[1] = ws.count(2)
WS1991[2] = ws.count(3)
WS1991[3] = ws.count(4)
WS1991[4] = ws.count(5)
WS1991[5] = ws.count(6)
WS1991[6] = ws.count(7)
WS1991[7] = ws.count(8)
WS1991[8] = ws.count(9)
WS1991[9] = ws.count(10)
WS1991[10] = ws.count(11)

WS1991x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1991x[0] = wsx.count(1)
WS1991x[1] = wsx.count(2)
WS1991x[2] = wsx.count(3)
WS1991x[3] = wsx.count(4)
WS1991x[4] = wsx.count(5)
WS1991x[5] = wsx.count(6)
WS1991x[6] = wsx.count(7)
WS1991x[7] = wsx.count(8)
WS1991x[8] = wsx.count(9)
WS1991x[9] = wsx.count(10)
WS1991x[10] = wsx.count(11)

print(WS1991, WS1991x)

[186. 656.  44.  25. 121.   8. 130.  92.  58.  26.   2.] [ 577. 2877.  348.  271. 1083.   68. 1824. 1499.  415.  189.   12.]


## 1992

In [341]:
#1992
a = -1  # counter for Jun
b = 123  # counter for Jul
c = 247  # counter for Aug

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [342]:
fn = r"WS\1992.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jun = ds.variables['june'][:]
jun6hr = jun[0::2]
jun6hr = np.ma.filled(jun6hr, fill_value=0)
juntranslated = np.roll(jun6hr, 180, axis=2)
junsize = np.size(jun6hr, 0)

jul = ds.variables['july'][:]
jul6hr = jul[0::2]
jul6hr = np.ma.filled(jul6hr, fill_value=0)
jultranslated = np.roll(jul6hr, 180, axis=2)
julsize = np.size(jul6hr, 0)

aug = ds.variables['august'][:]
aug6hr = aug[0::2]
aug6hr = np.ma.filled(aug6hr, fill_value=0)
augtranslated = np.roll(aug6hr, 180, axis=2)
augsize = np.size(aug6hr, 0)

In [343]:
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1992\JUN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1992\JUN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = juntranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = juntranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [344]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1992\JUL"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1992\JUL", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jultranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jultranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [345]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1992\AUG"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1992\AUG", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = augtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = augtranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [346]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [347]:
WS1992 = np.zeros(11)  # Count occurences of each WS for with DI
WS1992[0] = ws.count(1)
WS1992[1] = ws.count(2)
WS1992[2] = ws.count(3)
WS1992[3] = ws.count(4)
WS1992[4] = ws.count(5)
WS1992[5] = ws.count(6)
WS1992[6] = ws.count(7)
WS1992[7] = ws.count(8)
WS1992[8] = ws.count(9)
WS1992[9] = ws.count(10)
WS1992[10] = ws.count(11)

WS1992x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1992x[0] = wsx.count(1)
WS1992x[1] = wsx.count(2)
WS1992x[2] = wsx.count(3)
WS1992x[3] = wsx.count(4)
WS1992x[4] = wsx.count(5)
WS1992x[5] = wsx.count(6)
WS1992x[6] = wsx.count(7)
WS1992x[7] = wsx.count(8)
WS1992x[8] = wsx.count(9)
WS1992x[9] = wsx.count(10)
WS1992x[10] = wsx.count(11)

print(WS1992, WS1992x)

[ 90. 743.  18.  31. 124.   4. 192. 240.  43.  24.   2.] [ 372. 2648.  224.  151. 1221.  114. 1701. 1989.  541.  226.   17.]


## 1993

In [348]:
#1993
a = -1  # counter for Jun
b = 123  # counter for Jul
c = 247  # counter for Aug

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [349]:
fn = r"WS\1993.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jun = ds.variables['june'][:]
jun6hr = jun[0::2]
jun6hr = np.ma.filled(jun6hr, fill_value=0)
juntranslated = np.roll(jun6hr, 180, axis=2)
junsize = np.size(jun6hr, 0)

jul = ds.variables['july'][:]
jul6hr = jul[0::2]
jul6hr = np.ma.filled(jul6hr, fill_value=0)
jultranslated = np.roll(jul6hr, 180, axis=2)
julsize = np.size(jul6hr, 0)

aug = ds.variables['august'][:]
aug6hr = aug[0::2]
aug6hr = np.ma.filled(aug6hr, fill_value=0)
augtranslated = np.roll(aug6hr, 180, axis=2)
augsize = np.size(aug6hr, 0)

In [350]:
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1993\JUN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1993\JUN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = juntranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = juntranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [351]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1993\JUL"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1993\JUL", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jultranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jultranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [352]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1993\AUG"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1993\AUG", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = augtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = augtranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [353]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [354]:
WS1993 = np.zeros(11)  # Count occurences of each WS for with DI
WS1993[0] = ws.count(1)
WS1993[1] = ws.count(2)
WS1993[2] = ws.count(3)
WS1993[3] = ws.count(4)
WS1993[4] = ws.count(5)
WS1993[5] = ws.count(6)
WS1993[6] = ws.count(7)
WS1993[7] = ws.count(8)
WS1993[8] = ws.count(9)
WS1993[9] = ws.count(10)
WS1993[10] = ws.count(11)

WS1993x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1993x[0] = wsx.count(1)
WS1993x[1] = wsx.count(2)
WS1993x[2] = wsx.count(3)
WS1993x[3] = wsx.count(4)
WS1993x[4] = wsx.count(5)
WS1993x[5] = wsx.count(6)
WS1993x[6] = wsx.count(7)
WS1993x[7] = wsx.count(8)
WS1993x[8] = wsx.count(9)
WS1993x[9] = wsx.count(10)
WS1993x[10] = wsx.count(11)

print(WS1993, WS1993x)

[240. 713.  22.  31. 137.   3. 226. 241.  37.  22.   0.] [ 556. 3299.  442.  559. 1128.  148. 2907. 1944.  489.  139.   14.]


## 1994

In [355]:
1994
a = -1  # counter for Jun
b = 123  # counter for Jul
c = 247  # counter for Aug

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [356]:
fn = r"WS\1994.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jun = ds.variables['june'][:]
jun6hr = jun[0::2]
jun6hr = np.ma.filled(jun6hr, fill_value=0)
juntranslated = np.roll(jun6hr, 180, axis=2)
junsize = np.size(jun6hr, 0)

jul = ds.variables['july'][:]
jul6hr = jul[0::2]
jul6hr = np.ma.filled(jul6hr, fill_value=0)
jultranslated = np.roll(jul6hr, 180, axis=2)
julsize = np.size(jul6hr, 0)

aug = ds.variables['august'][:]
aug6hr = aug[0::2]
aug6hr = np.ma.filled(aug6hr, fill_value=0)
augtranslated = np.roll(aug6hr, 180, axis=2)
augsize = np.size(aug6hr, 0)

In [357]:
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1994\JUN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1994\JUN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = juntranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = juntranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [358]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1994\JUL"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1994\JUL", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jultranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jultranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [359]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1994\AUG"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1994\AUG", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = augtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = augtranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [360]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [361]:
WS1994 = np.zeros(11)  # Count occurences of each WS for with DI
WS1994[0] = ws.count(1)
WS1994[1] = ws.count(2)
WS1994[2] = ws.count(3)
WS1994[3] = ws.count(4)
WS1994[4] = ws.count(5)
WS1994[5] = ws.count(6)
WS1994[6] = ws.count(7)
WS1994[7] = ws.count(8)
WS1994[8] = ws.count(9)
WS1994[9] = ws.count(10)
WS1994[10] = ws.count(11)

WS1994x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1994x[0] = wsx.count(1)
WS1994x[1] = wsx.count(2)
WS1994x[2] = wsx.count(3)
WS1994x[3] = wsx.count(4)
WS1994x[4] = wsx.count(5)
WS1994x[5] = wsx.count(6)
WS1994x[6] = wsx.count(7)
WS1994x[7] = wsx.count(8)
WS1994x[8] = wsx.count(9)
WS1994x[9] = wsx.count(10)
WS1994x[10] = wsx.count(11)

print(WS1994, WS1994x)

[225. 787.  49.  22. 140.  12. 358. 214.  41.   6.   0.] [ 814. 3557.  304.  385.  935.   93. 2596. 1993.  404.  197.   34.]


## 1995

In [362]:
#1995
a = -1  # counter for Jun
b = 123  # counter for Jul
c = 247  # counter for Aug

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [363]:
fn = r"WS\1995.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jun = ds.variables['june'][:]
jun6hr = jun[0::2]
jun6hr = np.ma.filled(jun6hr, fill_value=0)
juntranslated = np.roll(jun6hr, 180, axis=2)
junsize = np.size(jun6hr, 0)

jul = ds.variables['july'][:]
jul6hr = jul[0::2]
jul6hr = np.ma.filled(jul6hr, fill_value=0)
jultranslated = np.roll(jul6hr, 180, axis=2)
julsize = np.size(jul6hr, 0)

aug = ds.variables['august'][:]
aug6hr = aug[0::2]
aug6hr = np.ma.filled(aug6hr, fill_value=0)
augtranslated = np.roll(aug6hr, 180, axis=2)
augsize = np.size(aug6hr, 0)

In [364]:
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1995\JUN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1995\JUN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = juntranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = juntranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [365]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1995\JUL"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1995\JUL", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jultranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jultranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [366]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1995\AUG"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1995\AUG", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = augtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = augtranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [367]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [368]:
WS1995 = np.zeros(11)  # Count occurences of each WS for with DI
WS1995[0] = ws.count(1)
WS1995[1] = ws.count(2)
WS1995[2] = ws.count(3)
WS1995[3] = ws.count(4)
WS1995[4] = ws.count(5)
WS1995[5] = ws.count(6)
WS1995[6] = ws.count(7)
WS1995[7] = ws.count(8)
WS1995[8] = ws.count(9)
WS1995[9] = ws.count(10)
WS1995[10] = ws.count(11)

WS1995x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1995x[0] = wsx.count(1)
WS1995x[1] = wsx.count(2)
WS1995x[2] = wsx.count(3)
WS1995x[3] = wsx.count(4)
WS1995x[4] = wsx.count(5)
WS1995x[5] = wsx.count(6)
WS1995x[6] = wsx.count(7)
WS1995x[7] = wsx.count(8)
WS1995x[8] = wsx.count(9)
WS1995x[9] = wsx.count(10)
WS1995x[10] = wsx.count(11)

print(WS1995, WS1995x)

[110. 437.  44.  21.  87.   3. 127. 126.  39.  16.   0.] [ 230. 2648.  512.  338. 1065.  176. 2615. 1724.  690.  260.   10.]


## 1996

In [369]:
#1996
a = -1  # counter for Jun
b = 123  # counter for Jul
c = 247  # counter for Aug

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [370]:
fn = r"WS\1996.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jun = ds.variables['june'][:]
jun6hr = jun[0::2]
jun6hr = np.ma.filled(jun6hr, fill_value=0)
juntranslated = np.roll(jun6hr, 180, axis=2)
junsize = np.size(jun6hr, 0)

jul = ds.variables['july'][:]
jul6hr = jul[0::2]
jul6hr = np.ma.filled(jul6hr, fill_value=0)
jultranslated = np.roll(jul6hr, 180, axis=2)
julsize = np.size(jul6hr, 0)

aug = ds.variables['august'][:]
aug6hr = aug[0::2]
aug6hr = np.ma.filled(aug6hr, fill_value=0)
augtranslated = np.roll(aug6hr, 180, axis=2)
augsize = np.size(aug6hr, 0)

In [371]:
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1996\JUN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1996\JUN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = juntranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = juntranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [372]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1996\JUL"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1996\JUL", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jultranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jultranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [373]:
a = -1
for filename in os.listdir("DI_FRONT\JJA1983-2004\JJA_1996\AUG"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\JJA1983-2004\JJA_1996\AUG", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = augtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = augtranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [374]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [375]:
WS1996 = np.zeros(11)  # Count occurences of each WS for with DI
WS1996[0] = ws.count(1)
WS1996[1] = ws.count(2)
WS1996[2] = ws.count(3)
WS1996[3] = ws.count(4)
WS1996[4] = ws.count(5)
WS1996[5] = ws.count(6)
WS1996[6] = ws.count(7)
WS1996[7] = ws.count(8)
WS1996[8] = ws.count(9)
WS1996[9] = ws.count(10)
WS1996[10] = ws.count(11)

WS1996x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1996x[0] = wsx.count(1)
WS1996x[1] = wsx.count(2)
WS1996x[2] = wsx.count(3)
WS1996x[3] = wsx.count(4)
WS1996x[4] = wsx.count(5)
WS1996x[5] = wsx.count(6)
WS1996x[6] = wsx.count(7)
WS1996x[7] = wsx.count(8)
WS1996x[8] = wsx.count(9)
WS1996x[9] = wsx.count(10)
WS1996x[10] = wsx.count(11)

print(WS1996, WS1996x)

[151. 690.  59.  73. 199.  21. 373. 203.  84.  30.  16.] [ 239. 2827.  374.  393. 1427.   66. 2454. 1506.  830.  324.   30.]


In [376]:
pd.DataFrame(data=[WS1984, WS1984x, WS1985, WS1985x, WS1986, WS1986x, WS1987, WS1987x, WS1988, WS1988x, WS1989, WS1989x, WS1990,\
    WS1990x, WS1991, WS1991x, WS1992, WS1992x, WS1993, WS1993x, WS1994, WS1994x, WS1995, WS1995x, WS1996, WS1996x],\
          index = ["1984 With DI"," 1984 W/O DIs","1985 With DI"," 1985 W/O DIs","1986 With DI"," 1986 W/O DIs","1987 With DI"," 1987 W/O DIs","1988 With DI",\
              " 1988 W/O DIs","1989 With DI"," 1989 W/O DIs","1990 With DI"," 1990 W/O DIs","1991 With DI"," 1991 W/O DIs","1992 With DI"," 1992 W/O DIs",\
                  "1993 With DI"," 1993 W/O DIs", "1994 With DI"," 1994 W/O DIs","1995 With DI"," 1995 W/O DIs","1996 With DI"," 1996 W/O DIs"], columns = ['1', '2', '3', '4', '5','6','7','8','9','10','11']).to_csv('JJA.csv')
